In [3]:
import os
import urllib
import shutil
import azureml

from azureml.core import Experiment
from azureml.core import Workspace, Run

from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
import logging
import azureml.dataprep as dprep 
from azureml.core import Dataset

In [2]:
data_df = pd.read_csv('data/ML_Training_data_Risk_Sales_v1.csv', low_memory=False)
test_df= pd.read_csv('data/ML_Testing_data_Risk_Sales_v1.csv', low_memory=False)

In [49]:
data_df.fillna(data_df.mean(), inplace=True)
test_df.fillna(data_df.mean(), inplace=True)

In [51]:
data_df.head()

,SALES_CHANNEL,TOTAL_PRICE,MAKE1,MODEL1,PRICE1,MAKE2,MODEL2,PRICE2,MAKE3,MODEL3,...,FRAUDSTATUSIND,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,Risk Label,START_TO_CLOSE_MINS
0,ONLINE,631.56,Samsung,Galaxy Note8,599.99,NaN,NaN,NaN,NaN,NaN,...,Green,19.51,10.77,233555.88,4091.53,4591.62,4040.29,0,Good,2421.68
1,TELESALES,126.96,Motorola,moto e5 play,119.99,NaN,NaN,NaN,NaN,NaN,...,Green,19.51,22.35,19.82,4091.53,4591.62,4040.29,0,Good,1634.05
2,TELESALES,1401.76,Apple,iPhone XS Max,1149.99,Case-Mate,Apple iPhone XS Max Case & GLASS Screen Protec...,44.95,NaN,NaN,...,Red,19.30,22.05,305.78,4330.95,2988.13,4292.47,1,Fraud,1342.82
3,TELESALES,0.00,Apple,iPhone 6s Plus,0,NaN,NaN,NaN,NaN,NaN,...,Green,19.51,36.85,9.83,4091.53,4591.62,4040.29,0,Good,1860.17
4,ONLINE,1337.42,Apple,iPhone XS Max,1149.99,NaN,NaN,NaN,NaN,NaN,...,Red,17.93,20.18,1025688.28,3650.92,4591.62,3626.18,1,Fraud,7083.53


In [52]:
data=data_df[['SALES_CHANNEL','TOTAL_PRICE','FINAL_RESULT','HOUR_OF_DAY','ACTIVE_WATCH_DURATION_MINS','ORDER_HOLD_DURATION_MINS','ACCOUNT_OPEN_TO_ORDER_MINS','CASECLOSE_TO_ORDERCLOSE_MINS','CASEOPEN_TO_CASECLOSE_MINS','Risk Label','START_TO_CLOSE_MINS']]
test_data=test_df[['SALES_CHANNEL','TOTAL_PRICE','FINAL_RESULT','HOUR_OF_DAY','ACTIVE_WATCH_DURATION_MINS','ORDER_HOLD_DURATION_MINS','ACCOUNT_OPEN_TO_ORDER_MINS','CASECLOSE_TO_ORDERCLOSE_MINS','CASEOPEN_TO_CASECLOSE_MINS','Risk Label','START_TO_CLOSE_MINS']]

In [53]:
data.shape

(96458, 11)

In [54]:
test_data.shape

(36997, 11)

In [55]:
data.isnull().values.any()
data.isnull().sum()

SALES_CHANNEL                   0
TOTAL_PRICE                     0
FINAL_RESULT                    0
HOUR_OF_DAY                     0
ACTIVE_WATCH_DURATION_MINS      0
ORDER_HOLD_DURATION_MINS        0
ACCOUNT_OPEN_TO_ORDER_MINS      0
CASECLOSE_TO_ORDERCLOSE_MINS    0
CASEOPEN_TO_CASECLOSE_MINS      0
Risk Label                      0
START_TO_CLOSE_MINS             0
dtype: int64

In [56]:
test_data.isnull().sum()

SALES_CHANNEL                   0
TOTAL_PRICE                     0
FINAL_RESULT                    0
HOUR_OF_DAY                     0
ACTIVE_WATCH_DURATION_MINS      0
ORDER_HOLD_DURATION_MINS        0
ACCOUNT_OPEN_TO_ORDER_MINS      0
CASECLOSE_TO_ORDERCLOSE_MINS    0
CASEOPEN_TO_CASECLOSE_MINS      0
Risk Label                      0
START_TO_CLOSE_MINS             0
dtype: int64

In [34]:
#test_data['ACTIVE_WATCH_DURATION_MINS'].fillna('NA',inplace=True)
#test_data['CASECLOSE_TO_ORDERCLOSE_MINS'].fillna('NA',inplace=True)
#test_data['CASEOPEN_TO_CASECLOSE_MINS'].fillna('NA',inplace=True)
#test_data['START_TO_CLOSE_MINS'].fillna('NA',inplace=True)

In [57]:
test_data.head()

,SALES_CHANNEL,TOTAL_PRICE,FINAL_RESULT,HOUR_OF_DAY,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,Risk Label,START_TO_CLOSE_MINS
0,TELESALES,847.92,YELLOW,16,18.65,21.18,7.83,4591.62,2820.68,Fraud,7083.53
1,TELESALES,0.00,GREEN,23,20.32,93.03,8.30,4591.62,4040.29,Good,11056.48
2,TELESALES,126.96,GREEN,1,19.51,22.65,1.77,4591.62,4040.29,Good,8048.73
3,RETAIL,655.26,GREEN,19,19.51,10.97,1467.63,4591.62,4040.29,Good,7060.18
4,ONLINE,479.57,GREEN,11,18.95,43.45,59000.28,4591.62,4040.29,Good,4889.35


In [58]:
def estimated(row):
    duration = row['START_TO_CLOSE_MINS']
#     print(color)
    if duration <= 2999 :
        return 0
    elif duration >2999 and duration <=6999 :
        return 1
    elif duration >6999 and duration <=10999 :
        return 2
    elif duration >10999 and duration <=20999 :
        return 3
    else:
        return 4

In [59]:
data['label'] = data.apply(estimated, axis=1)
data.drop(columns=['START_TO_CLOSE_MINS'], inplace=True)

test_data['label'] = test_data.apply(estimated, axis=1)
test_data.drop(columns=['START_TO_CLOSE_MINS'], inplace=True)

In [60]:
data.head()

,SALES_CHANNEL,TOTAL_PRICE,FINAL_RESULT,HOUR_OF_DAY,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,Risk Label,label
0,ONLINE,631.56,GREEN,0,19.51,10.77,233555.88,4591.62,4040.29,Good,0
1,TELESALES,126.96,GREEN,19,19.51,22.35,19.82,4591.62,4040.29,Good,0
2,TELESALES,1401.76,RED,23,19.30,22.05,305.78,2988.13,4292.47,Fraud,0
3,TELESALES,0.00,RED,14,19.51,36.85,9.83,4591.62,4040.29,Good,0
4,ONLINE,1337.42,RED,12,17.93,20.18,1025688.28,4591.62,3626.18,Fraud,2


In [68]:
test_data['label'].max()

3

In [62]:
collections.Counter(data['label'])

Counter({0: 18123, 2: 48472, 1: 28592, 3: 1271})

In [63]:
twoday_df = data[data['label'] == 0]
fourday_df = data[data['label'] == 1]
sevenday_df = data[data['label'] == 2]
twoweekday_df = data[data['label'] == 3]
stallday_df = data[data['label'] == 4]

In [71]:
twoday_under_df = twoday_df.sample(28592, replace=True)
sevenday_over_df= sevenday_df.sample(28592, replace=True)
twoweekday_under_df = twoweekday_df.sample(28592, replace=True)

In [73]:
balanced_data_df = pd.concat([twoday_under_df,sevenday_over_df,twoweekday_under_df])

In [74]:
balanced_data_df.shape

(85776, 11)

In [75]:
X_train = balanced_data_df.drop(columns=['label'])
Y_train = balanced_data_df['label']

X_test = test_data.drop(columns=['label'])
Y_test = test_data['label']

In [76]:
X_train.shape

(85776, 10)

In [77]:
Y_train = np.array(Y_train)

In [78]:
Y_train.shape

(85776,)

In [79]:
X_test.shape

(36997, 10)

In [80]:
Y_test.shape

(36997,)

In [81]:
balanced_data_df.to_csv('data/EstimatedShipping_train.csv',index=False)
test_data.to_csv('data/EstimatedShipping_test.csv',index=False)

In [82]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'EstimatedShipping'
# project folder
project_folder = './ml_experiment/EstimatedShipping'
 
experiment = Experiment(ws, experiment_name)
 
output = {}
output['SDK version'] = azureml.core.VERSION
output['Subscription ID'] = ws.subscription_id
output['Workspace Name'] = ws.name
output['Resource Group'] = ws.resource_group
output['Location'] = ws.location
output['Project Directory'] = project_folder
output['Experiment Name'] = experiment.name
pd.set_option('display.max_colwidth', -1)
outputDf = pd.DataFrame(data = output, index = [''])
outputDf.T

,
SDK version,1.0.69
Subscription ID,be564fde-136b-4709-b7b6-abfc0bdfc134
Workspace Name,xm-ml-workspace
Resource Group,xm-ml
Location,centralus
Project Directory,./ml_experiment/EstimatedShipping
Experiment Name,EstimatedShipping


In [85]:
train_dflow = dprep.read_csv('data/EstimatedShipping_train.csv', infer_column_types=True)
train_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
SALES_CHANNEL,FieldType.STRING,ONLINE,TELESALES,85776.00,0.00,85776.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
TOTAL_PRICE,FieldType.DECIMAL,0.00,7992.25,85776.00,0.00,85776.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,49.88,483.67,811.06,1530.85,2637.08,4826.40,556.60,576.09,331881.64,2.34,12.46,1895.82,0.00
FINAL_RESULT,FieldType.STRING,GREEN,YELLOW,85776.00,0.00,85776.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
HOUR_OF_DAY,FieldType.INTEGER,0.00,23.00,85776.00,0.00,85776.00,0.00,0.00,0.00,24,0.00,1.99,1.99,14.00,17.00,20.00,23.00,23.00,23.00,15.42,6.84,46.82,-1.20,0.22,23.00,5.95
ACTIVE_WATCH_DURATION_MINS,FieldType.DECIMAL,8.50,118.28,85776.00,0.00,85776.00,0.00,0.00,0.00,919,13.18,16.87,16.81,19.51,19.51,19.51,23.03,25.02,27.90,19.45,2.08,4.34,3.58,147.08,19.51,19.51
ORDER_HOLD_DURATION_MINS,FieldType.DECIMAL,4.23,1200.02,85776.00,0.00,85776.00,0.00,0.00,0.00,>1000,5.32,10.47,10.35,13.85,19.19,33.06,115.08,225.30,437.27,31.86,45.16,2039.20,5.61,55.03,60.00,4.23
ACCOUNT_OPEN_TO_ORDER_MINS,FieldType.DECIMAL,0.03,1492315.72,85776.00,0.00,85776.00,0.00,0.00,0.00,>1000,0.31,2.55,2.51,7.13,869.66,127195.85,803983.26,1035064.52,1135228.32,132331.86,256700.54,65895166166.45,2.16,3.78,315867.81,0.03
CASECLOSE_TO_ORDERCLOSE_MINS,FieldType.DECIMAL,0.95,162474.42,85776.00,0.00,85776.00,0.00,0.00,0.00,>1000,120.48,4591.62,4591.62,4591.62,4591.62,4591.62,5362.22,28016.18,41795.98,5300.62,5048.01,25482359.52,14.26,342.66,4591.62,4591.62
CASEOPEN_TO_CASECLOSE_MINS,FieldType.DECIMAL,61.13,6494.42,85776.00,0.00,85776.00,0.00,0.00,0.00,>1000,1459.39,4040.29,4040.29,4040.29,4040.29,4040.29,4135.09,6480.03,6480.07,4019.75,562.97,316940.16,0.25,11.65,4040.29,4040.29
Risk Label,FieldType.STRING,Collections,Good,85776.00,0.00,85776.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,


In [86]:
test_dflow = dprep.read_csv('data/EstimatedShipping_test.csv', infer_column_types=True)
test_dflow.get_profile()

,Type,Min,Max,Count,Missing Count,Not Missing Count,Percent Missing,Error Count,Empty Count,Unique Values,0.1% Quantile (est.),1% Quantile (est.),5% Quantile (est.),25% Quantile (est.),50% Quantile (est.),75% Quantile (est.),95% Quantile (est.),99% Quantile (est.),99.9% Quantile (est.),Mean,Standard Deviation,Variance,Skewness,Kurtosis,WhiskerTop,WhiskerBottom
SALES_CHANNEL,FieldType.STRING,ONLINE,TELESALES,36997.00,0.00,36997.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
TOTAL_PRICE,FieldType.DECIMAL,0.00,6890.15,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,0.00,0.00,0.00,0.00,378.50,886.31,1642.45,2668.87,5043.41,559.61,644.53,415419.57,1.97,8.05,2153.59,0.00
FINAL_RESULT,FieldType.STRING,GREEN,YELLOW,36997.00,0.00,36997.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,
HOUR_OF_DAY,FieldType.INTEGER,0.00,23.00,36997.00,0.00,36997.00,0.00,0.00,0.00,24,0.00,1.60,1.00,13.98,17.00,20.00,23.00,23.00,23.00,15.16,6.94,48.14,-1.11,0.01,23.00,5.79
ACTIVE_WATCH_DURATION_MINS,FieldType.DECIMAL,7.50,29.92,36997.00,0.00,36997.00,0.00,0.00,0.00,869,14.73,19.51,19.51,19.51,19.51,23.08,27.08,28.54,29.44,21.25,2.81,7.92,1.02,0.11,28.41,15.72
ORDER_HOLD_DURATION_MINS,FieldType.DECIMAL,1.42,1288.20,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,10.19,14.64,14.48,18.52,24.11,32.04,158.83,290.71,389.75,42.97,54.30,2948.91,3.56,19.84,49.12,1.42
ACCOUNT_OPEN_TO_ORDER_MINS,FieldType.DECIMAL,0.03,1549937.45,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,0.42,2.52,2.46,6.35,1283.73,121671.44,881253.15,1106461.08,1230996.89,143003.60,279419.90,78075477987.57,2.16,3.69,301931.19,0.03
CASECLOSE_TO_ORDERCLOSE_MINS,FieldType.DECIMAL,110.35,82270.10,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,1398.09,4591.62,4591.62,4591.62,4591.62,4591.62,4591.68,9065.44,18388.02,4694.97,1300.07,1690177.02,24.70,1168.95,4591.62,4591.62
CASEOPEN_TO_CASECLOSE_MINS,FieldType.DECIMAL,1444.33,6482.22,36997.00,0.00,36997.00,0.00,0.00,0.00,>1000,1490.65,4040.29,4040.29,4040.29,4040.29,4040.29,4283.81,6480.03,6480.07,4047.96,600.85,361018.28,1.15,10.05,4040.29,4040.29
Risk Label,FieldType.STRING,Collections,Good,36997.00,0.00,36997.00,0.00,0.00,0.00,3,,,,,,,,,,,,,,,,


In [87]:
X_df = train_dflow.drop_columns(columns=['label'])
y_df = train_dflow.keep_columns(columns=['label'], validate_column_exists=True)
X_test_df = test_dflow.drop_columns(columns=['label'])
y_test_df = test_dflow.keep_columns(columns=['label'], validate_column_exists=True)

In [88]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

cts = ws.compute_targets
amlcompute_cluster_name = "XM-Heavy-Load"
compute_target = cts[amlcompute_cluster_name]



found = False

# # Check if this compute target already exists in the workspace.

cts = ws.compute_targets
if amlcompute_cluster_name in cts and cts[amlcompute_cluster_name].type == 'AmlCompute':
    found = True
    print('Found existing compute target.')
    compute_target = cts[amlcompute_cluster_name]

if not found:
    print('Creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size = "STANDARD_D2_V2", # for GPU, use "STANDARD_NC6"
                                                                vm_priority = 'lowpriority',
                                                                min_nodes = 0, max_nodes = 6)

    # Create the cluster.\n",
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, provisioning_config)

#     # Can poll for a minimum number of nodes and for a specific timeout.
#     # If no min_node_count is provided, it will use the scale settings for the cluster.
#     compute_target.wait_for_completion(show_output = True, min_node_count = None, timeout_in_minutes = 20)

#      # For a more detailed view of current AmlCompute status, use get_status().

Found existing compute target.


In [89]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
import pkg_resources

# create a new RunConfig object
conda_run_config = RunConfiguration(framework="python")

# Set compute target to AmlCompute
conda_run_config.target = compute_target
conda_run_config.environment.docker.enabled = True
conda_run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE

# dprep_dependency = 'azureml-dataprep==' + pkg_resources.get_distribution("azureml-dataprep").version

cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', 'psutil'], conda_packages=['numpy','py-xgboost<=0.80'])
# cd = CondaDependencies.create(pip_packages=['azureml-sdk[automl]', dprep_dependency], conda_packages=['numpy','py-xgboost<=0.80'])
conda_run_config.environment.python.conda_dependencies = cd

In [91]:
# Dump X and Y to CSVs.
X_df.to_pandas_dataframe().to_csv('data/shipment/X_train.csv', index=False)
y_df.to_pandas_dataframe().to_csv('data/shipment/Y_train.csv', index=False)

# Dump X and Y to CSVs.
X_test_df.to_pandas_dataframe().to_csv('data/shipment/X_test.csv', index=False)
y_test_df.to_pandas_dataframe().to_csv('data/shipment/Y_test.csv', index=False)

In [93]:
ds = ws.get_default_datastore()
ds.upload(src_dir='./data/shipment', target_path='data/shipment', overwrite=True, show_progress=True)

Uploading an estimated of 4 files
Uploading ./data/shipment/X_test.csv
Uploading ./data/shipment/X_train.csv
Uploading ./data/shipment/Y_test.csv
Uploading ./data/shipment/Y_train.csv
Uploaded ./data/shipment/Y_train.csv, 1 files out of an estimated total of 4
Uploaded ./data/shipment/Y_test.csv, 2 files out of an estimated total of 4
Uploaded ./data/shipment/X_test.csv, 3 files out of an estimated total of 4
Uploaded ./data/shipment/X_train.csv, 4 files out of an estimated total of 4
Uploaded 4 files


$AZUREML_DATAREFERENCE_94ebaedd30c74810815e38d799478ab2

In [94]:
X = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/X_train.csv'))
y = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/Y_train.csv'))

X_valid = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/X_test.csv'))
y_valid = Dataset.Tabular.from_delimited_files(path=ds.path('data/shipment/Y_test.csv'))

In [95]:
automl_config = AutoMLConfig(task = 'classification',
                             primary_metric = 'AUC_weighted',
                             debug_log='automl_errors.log',
                             path = project_folder, #For external compute
                             run_configuration=conda_run_config, #For external compute
                             preprocess=True,
                             X = X, 
                             y = y,
                             X_valid=X_valid,
                             y_valid=y_valid,
                             iteration_timeout_minutes=10,
                             experiment_timeout_minutes=200,
                             max_concurrent_iterations=10,
                             max_cores_per_iteration=4,
                             iterations=100,
                             model_explainability=True,
                             )

In [ ]:
# New Lines
# conda_env.docker.enabled = True
# conda_env.python.user_managed_dependencies = False

# print('Base Docker image is:', conda_env.docker.base_image)
# END  New Lines
ws = Workspace.from_config()

experiment_name = 'EstimatedShipping'

experiment = Experiment(ws, experiment_name)
#local_run = experiment.submit(automl_config, show_output = True)
remote_run = experiment.submit(automl_config, show_output = True)

Running on remote or ADB.
Running on remote compute: XM-Heavy-Load
Parent Run ID: AutoML_f96f2d47-4cd1-4333-b83b-630f466db848
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
ITERATION: The iteration being evaluated.
PIPELINE: A summary description of the pipeline being evaluated.
DURATION: Time taken for the current iteration.
METRIC: The result of computing score on the fitted pipeline.
BEST: The best observed score thus far.
****************************************************************************************************

 ITERATION   PIPELINE                                       DURATION      METRIC      BEST
         0   MaxAbsScaler LightGBM                          0:01:37       0.5538    0.5538
         1   MaxA

In [102]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
EstimatedShipping,AutoML_f96f2d47-4cd1-4333-b83b-630f466db848,automl,Completed,Link to Azure Portal,Link to Documentation


In [103]:
best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

Run(Experiment: EstimatedShipping,
Id: AutoML_f96f2d47-4cd1-4333-b83b-630f466db848_99,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
     steps=[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
        feature_sweeping_config=None, feature_sweeping_timeout=None,
        featurization_config=None, is_cross_validation=None,
        is_onnx_compatible=None, logger=None, observer=None, task=None)), ('stac...7f6471e4b7f0>,
           solver='lbfgs', tol=0.0001, verbose=0),
            training_cv_folds=5))])


In [104]:
print(overall_summary)
print(overall_imp)
print(per_class_summary)
print(per_class_imp)

[82581.17936681156, 28299.876667826702, 23003.752118361208, 3126.1658203002426, 2749.649595197979, 2258.1664907520785, 2003.4724845862154, 778.5107854115246, 500.4728878466681, 406.5297185422721, 199.26780028763383, 78.29088938427327, 29.454670048807024, 28.872182625867236, 20.801320179979857, 18.888787751743532, 13.709129271459258, 8.342154709661779, 6.093765385404226, 4.321086212691751, 4.012299941585305, 3.394367560749088, 2.8607555490374907, 2.500973917723351, 2.263839273729871, 1.9704602704528806, 1.2559231778689492, 0.8252659086091907, 0.6742364855422113, 0.46516778837147416, 0.12032639336661315, 0.1140854160124586, 0.09040289501261299, 0.04589089941674802, 0.036379934710778696, 0.03206845695312715, 0.023949832016430965, 0.016634767132499447, 0.0028239805799601314]
['ORDER_HOLD_DURATION_MINS_MeanImputer', 'ACTIVE_WATCH_DURATION_MINS_MeanImputer', 'TOTAL_PRICE_MeanImputer', 'HOUR_OF_DAY_CharGramCountVectorizer_22', 'FINAL_RESULT_CharGramCountVectorizer_red', 'HOUR_OF_DAY_CharGramC